In [3]:
import pandas as pd
import matplotlib
matplotlib.use('TkAgg') 
from sklearn import preprocessing
from sklearn.metrics import pairwise
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans, AgglomerativeClustering  
import matplotlib.pylab as plt
import seaborn as sns
from pandas.plotting import parallel_coordinates
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import datetime

df = pd.read_csv('CasusData.csv')
df = df.copy()
print(df)  #show all the data

       WAP001  WAP002  WAP003  WAP004  WAP005  WAP006  WAP007  WAP008  WAP009  \
0         100     100     100     100     100     100     100     100     100   
1         100     100     100     100     100     100     100     100     100   
2         100     100     100     100     100     100     100     -97     100   
3         100     100     100     100     100     100     100     100     100   
4         100     100     100     100     100     100     100     100     100   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
19932     100     100     100     100     100     100     100     100     100   
19933     100     100     100     100     100     100     100     100     100   
19934     100     100     100     100     100     100     100     100     100   
19935     100     100     100     100     100     100     100     100     100   
19936     100     100     100     100     100     100     100     100     100   

       WAP010  ...  WAP520 

In [4]:
# Dit deel laat zien wanneer de users op de locatie waren van het evenement
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], unit='s')

# Groepeert de data in dagen
daily_user_ids = df.groupby(df['TIMESTAMP'].dt.floor('H'))['USERID'].unique()

# Creëert een dictionary voor iedere user met een kleur
user_id_color_map = {user_id: plt.cm.tab20(i % 20) for i, user_id in enumerate(df['USERID'].unique())}

# Plot de grafiek
for date, user_ids in daily_user_ids.items():
    colors = [user_id_color_map[user_id] for user_id in user_ids]
    plt.scatter([date] * len(user_ids), user_ids, alpha=0.7, c=colors, marker='o')

plt.yticks(range(int(min(df['USERID'])), int(max(df['USERID'])) + 1))
plt.xlabel('TIMESTAMP')
plt.ylabel('USERID')
plt.title('Active USERID per Hour')

plt.show()



In [5]:
# Maak een lege dictionary om de relatie tussen gebruikers en kleuren bij te houden
user_colors = {}

# Schakelt de interactieve modus in
plt.ion()

# Maakt een 3D-plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

begindatum = datetime.datetime(2013, 5, 30, 0, 0, 0)  
einddatum = datetime.datetime(2013, 6, 20, 0, 0, 0) 

begintijd_unix = int(begindatum.timestamp())
eindtijd_unix = int(einddatum.timestamp())

# Loop door elke verdieping
for FLOOR in sorted(df['FLOOR'].unique()):
    
    # Convert Unix timestamps to datetime objects
    begindatum = datetime.datetime.fromtimestamp(begintijd_unix)
    einddatum = datetime.datetime.fromtimestamp(eindtijd_unix)

    # Filter the dataset based on the time range
    df = df[(df['TIMESTAMP'] < begindatum) | (df['TIMESTAMP'] > einddatum)]
    
    # Filtert de dataset voor de huidige verdieping
    FLOOR_data = df[df['FLOOR'] == FLOOR]
    
    # Sorteert de data op basis van TIMESTAMP
    FLOOR_data = FLOOR_data.sort_values(by='TIMESTAMP')
    
    # Plot de locaties van alle gebruikers per verdieping
    for user_id in FLOOR_data['USERID'].unique():
        user_data = FLOOR_data[FLOOR_data['USERID'] == user_id]
        
        # Haalt de kleur op voor de gebruiker (en sla deze op als deze nog niet bestaat)
        if user_id not in user_colors:
            user_colors[user_id] = plt.cm.tab20(len(user_colors))
        
        # Plot de locaties in de 3D-plot
        ax.plot(user_data['LONGITUDE'], user_data['LATITUDE'], zs=FLOOR, marker='o', linestyle='-', color=user_colors[user_id], label=f'User {user_id}')
        
        # Verbindt de locaties tussen aangrenzende verdiepingen
        if FLOOR < max(df['FLOOR'].unique()):
            next_floor = FLOOR + 1
            next_floor_data = df[(df['FLOOR'] == next_floor) & (df['USERID'] == user_id)]
            if not next_floor_data.empty:
                # Verbindt de locaties tussen de huidige en volgende verdieping
                ax.plot([user_data['LONGITUDE'].iloc[-1], next_floor_data['LONGITUDE'].iloc[0]], 
                        [user_data['LATITUDE'].iloc[-1], next_floor_data['LATITUDE'].iloc[0]], 
                        [FLOOR, next_floor], linestyle='-', color=user_colors[user_id])


ax.set_title('Gebruikerslocaties per verdieping in 3D')
ax.set_xlabel('LONGITUDE')
ax.set_ylabel('LATITUDE')
ax.set_zlabel('FLOOR')

# Legenda buiten de plot plaatsen, want stond eerst op de z-as waardoor die niet te lezen was
ax_leg = fig.add_axes([0.85, 0.1, 0.05, 0.8])
handles = []
labels = []
for user_id, color in sorted(user_colors.items()):
    handles.append(plt.Line2D([0], [0], linestyle='-', marker='o', color=color))
    labels.append(f'User {user_id}')

ax_leg.legend(handles, labels, loc='center left', title='Legend')
ax_leg.axis('off')

plt.show()